In [1]:
import os

import numpy as np
from tensorflow import keras
import cv2

import matplotlib.pyplot as plt
import ipympl

from tqdm import tqdm
import image_handler as handler

In [2]:
model = keras.models.load_model("droplet_detection_model")

In [3]:
folder = "training_data"
# folder = os.path.join("output_data", "basic")

images = []
for file in os.listdir(folder):
    if file.endswith(".jpg"):
        images.append(file)

images = sorted(images)

# Compare Neural network output with CV2 fit 

In [68]:
pbar = tqdm(enumerate(images))

num_circles_detected = 0

for idx, image in pbar:
    pbar.set_description(image)    
    img = cv2.cvtColor(cv2.imread(os.path.join("training_data", image)), cv2.COLOR_BGR2GRAY)
    processed = img.copy()
    
    # processed = cv2.Canny(processed, 5, 70, 3)
    #smooth to reduce noise a bit more
    processed = cv2.GaussianBlur(processed, (5, 5), 0)
    
    circles = cv2.HoughCircles(processed, cv2.HOUGH_GRADIENT, 1, 50, param1=30, param2=30, minRadius=15, maxRadius=60)
    
    if circles is not None:
        num_circles_detected += 1
        
        circles = np.round(circles[0, :]).astype("int")
        
        for (x, y, r) in circles:
            cv2.circle(processed, center=(x, y), radius=r, color=(255, 255, 0), thickness=4)
            cv2.rectangle(processed, pt1=(x-5, y-5), pt2=(x+5, y+5), color=(0, 128, 255), thickness=-1)

    # cv2.imwrite(os.path.join("output_data", "circle_fitting", image), output)

    fig, ax = plt.subplots(ncols=2, figsize=(15, 8), sharex="all", sharey="all")
    
    ax[0].imshow(img, cmap="gray")
    ax[0].set_title("Neural Network Output")
    ax[1].imshow(processed, cmap="gray")
    ax[1].set_title("CV2 Circle Fit")
    
    fig.savefig(os.path.join("output_data", "droplet_detection", image), dpi=150, bbox_inches="tight")
    plt.close("all")

t9_well3_4.jpg: : 102it [00:31,  3.27it/s] 


In [71]:
print(f"Percent detected {num_circles_detected / len(images) * 100:.1f}%")

Percent detected 72.5%


# Overlay result on original image

In [83]:
pbar = tqdm(enumerate(images))

num_circles_detected = 0

for idx, image in pbar:
    pbar.set_description(image) 
    
    img_original, _, _= handler.load_image(
        os.path.join("training_data", image), 
        circle_label=None, 
        overlay=False, 
        scale_percent=30
    )
    
    img = cv2.cvtColor(cv2.imread(os.path.join("output_data", "basic", image)), cv2.COLOR_BGR2GRAY)

    processed = img.copy()
    processed = cv2.GaussianBlur(processed, (5, 5), 0)
    
    circles = cv2.HoughCircles(processed, cv2.HOUGH_GRADIENT, 1, 50, param1=30, param2=30, minRadius=15, maxRadius=60)
    
    if circles is not None:
        num_circles_detected += 1
        
        circles = np.round(circles[0, :]).astype("int")
        
        for (x, y, r) in circles:
            cv2.circle(img_original, center=(x, y), radius=r, color=(255, 255, 0), thickness=4)
            cv2.rectangle(processed, pt1=(x-5, y-5), pt2=(x+5, y+5), color=(0, 128, 255), thickness=-1)

    # cv2.imwrite(os.path.join("output_data", "circle_fitting", image), output)

    fig, ax = plt.subplots(figsize=(8, 6))
    ax.imshow(img_original, cmap="gray")
    ax.set_title("Neural Network Output")
    
    fig.savefig(os.path.join("output_data", "droplet_detection", image), dpi=150, bbox_inches="tight")
    plt.close("all")

t9_well3_4.jpg: : 102it [00:17,  5.79it/s] 


In [84]:
print(f"Percent detected {num_circles_detected / len(images) * 100:.1f}%")

Percent detected 72.5%


# Save basic images (no axes etc.)

In [7]:
pbar = tqdm(enumerate(images))
for idx, image in pbar:
    pbar.set_description(image)
    img, _ = handler.load_test_image(os.path.join(folder, image), circle_label=None, scale_percent=100)
    pred = model.predict(img)
    cv2.imwrite(os.path.join("output_data", "two_run", image), pred[0, :, :, :] * 255)

t9_well3_4.jpg: : 102it [00:02, 38.17it/s] 


# Compare with input images

In [4]:
pbar = tqdm(enumerate(images))
for idx, image in pbar:
    pbar.set_description(image)

    fig, ax = plt.subplots(ncols=2, sharex="all", sharey="all", figsize=(10, 7))

    img, _ = handler.load_test_image(os.path.join("training_data", image), circle_label=None)
    pred = model.predict(img)

    ax[0].imshow(img[0, :, :, :], cmap="gray")
    ax[0].set_title("Experimental Data")
    ax[1].imshow(pred[0, :, :, :], cmap="gray")
    ax[1].set_title("Neural Network")
    
    fig.savefig(os.path.join("output_data", "compare", image), dpi=150, bbox_inches="tight")
    plt.close("all")

t0_well0_0.jpg: : 0it [00:00, ?it/s]2022-04-26 23:40:19.581365: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
t9_well3_4.jpg: : 102it [00:15,  6.69it/s] 


# General model layout

In [ ]:
keras.utils.plot_model(model, to_file=os.path.join("results", "model.png"), show_shapes=True, show_layer_names=True)